In [1]:
from gensim.models import Word2Vec
import MeCab
def tokenize(text):
    """ テキストを形態素解析して、トークンのリストを返す """
    mecab = MeCab.Tagger("-Owakati")
    return mecab.parse(text).strip().split()
with open('./Data/dokujo-tsushin.txt', 'r', encoding='utf-8') as file:
    corpus = [line.strip() for line in file if line.strip()]  # 空白行をスキップ


In [2]:
tokenized_documents = [tokenize(doc) for doc in corpus]

In [3]:
model_jp = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4)

In [4]:
model_jp.wv['日本']

array([-0.42390206,  1.1417792 ,  0.39490527,  0.01861866, -0.0580457 ,
       -1.0609587 ,  0.35490078,  0.698109  , -0.99512666, -0.77484024,
       -0.61340576, -0.7312136 ,  0.6898073 ,  0.03087586, -0.3631231 ,
       -0.6893097 ,  0.31434277, -0.7462328 , -1.0427057 ,  0.05590496,
       -0.36251545,  0.47453773, -0.04416125, -0.5120807 ,  0.5361824 ,
        0.19517706, -0.26864764, -0.25809255, -0.50550985, -0.36159402,
       -0.82905704, -0.02751032,  0.27843806, -0.06855041, -0.41776097,
        0.3799754 , -0.07990406, -0.5167496 ,  0.52756983, -0.26479518,
       -0.4523949 , -0.05233534,  0.3776077 , -0.5340258 ,  0.7052093 ,
        0.00602448, -1.0807167 , -0.2795185 ,  0.49017465,  0.40885422,
        0.03972309, -0.501853  , -0.5518711 , -0.1882381 , -1.1514312 ,
        0.1751215 ,  0.8482001 ,  0.15459247, -0.04581664, -0.10349711,
        0.60894406,  0.38792682, -0.09221508,  0.3402899 , -0.5933291 ,
        1.1737163 , -0.36427304, -0.12396126, -0.03914814,  0.38

In [5]:
model_jp.wv.most_similar("男")

[('女', 0.9268308877944946),
 ('友達', 0.8252323865890503),
 ('男性', 0.7798293232917786),
 ('女の子', 0.7449547648429871),
 ('モテ', 0.7428614497184753),
 ('絶対', 0.7310413718223572),
 ('モテる', 0.7299210429191589),
 ('本当', 0.7271409630775452),
 ('彼女', 0.7237347364425659),
 ('恋人', 0.7231690883636475)]

In [6]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))
model = gensim.downloader.load('word2vec-google-news-300')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [7]:
# ベクトル演算を行う
result_vector = model['king'] - model['man'] + model['woman']

# 類似度の高い10語を取得
similar_words = model.most_similar(positive=[result_vector], topn=10)

# 結果を出力
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

king: 0.8449
queen: 0.7301
monarch: 0.6455
princess: 0.6156
crown_prince: 0.5819
prince: 0.5777
kings: 0.5614
sultan: 0.5377
Queen_Consort: 0.5344
queens: 0.5290


In [8]:

# ベクトル演算を行う
result_vector = model.wv['king'] - model.wv['man'] + model.wv['woman']

# 類似度の高い10語を取得
similar_words = model.wv.most_similar(positive=[result_vector], topn=10)

# 結果を出力
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [ ]:
male_list=["man","men","his","his","he","male","masculine"]
female_list=["woman","women","her","hers","she","female","feminine"]

In [ ]:
import numpy as np
male_vec=[]
for i,j in zip(male_list,female_list):
    male_vec.append(model[i]-model[j])
male_vec=np.array(male_vec)
male_vec=np.mean(male_vec,axis=0)

In [ ]:
for sport in sports:
    print(sport,get_angle(model[sport],male_vec,degree=True))

In [25]:
import numpy as np
from gensim.downloader import load
from numpy.linalg import norm

# モデルの読み込み
model = load("word2vec-google-news-300")

# 性別を代表する単語リスト
female_words = ["she", "female", "woman", "girl"]
male_words = ["he", "male", "man", "boy"]

# 職業単語リスト
occupation_words = ["engineer", "nurse", "housekeeper"]

# 性別バイアスを計算する関数
def calculate_gender_bias(occupation, female_words, male_words, model):
    # 距離の計算 (女性)
    female_distances = [norm(model[occupation] - model[female]) for female in female_words]
    female_mean = np.mean(female_distances)

    # 距離の計算 (男性)
    male_distances = [norm(model[occupation] - model[male]) for male in male_words]
    male_mean = np.mean(male_distances)

    # 性別バイアス
    gender_bias = female_mean - male_mean
    return gender_bias

# 各職業の性別バイアスを計算
bias_results = {}
for occupation in occupation_words:
    bias = calculate_gender_bias(occupation, female_words, male_words, model)
    bias_results[occupation] = bias



In [24]:
# 結果を表示
for occupation, bias in bias_results.items():
    relation = "closer to women" if bias < 0 else "closer to men"
    print(f"Gender Bias for {occupation}: {bias:.4f} ({relation})")

Gender Bias for engineer: 0.2160 (closer to men)
Gender Bias for nurse: -0.2625 (closer to women)
Gender Bias for housekeeper: -0.1960 (closer to women)
